# PATH

구글 드라이브

In [ ]:
import os
import sys
from google.colab import drive

drive.mount('/content/drive')
# 지금 RNN이란 폴더에서 작업해서 RNN으로 맞춘 상태입니다! 
os.chdir('/content/drive/My Drive/RNN')

Mounted at /content/drive


In [ ]:
os.getcwd()

'/content/drive/My Drive/RNN'

In [ ]:
root_data = f"{os.getcwd()}/data/data_hw"
root_data

'/content/drive/My Drive/RNN/data/data_hw'

In [ ]:
# 데이터 불러올 때 사용할 경로로 이후에 있을 unzip과 연동되는 내용입니다! 
path_train_data = "data/data_hw/1.Training/원천데이터/일상생활및구어체_영한_train_set.json"

path_val_data = "data/data_hw/2.Validation/원천데이터/일상생활및구어체_영한_valid_set.json"


# 사전 준비 사항

In [1]:
import os
import sys
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
os.chdir('/content/drive/My Drive/Data')

os.getcwd()

'/content/drive/My Drive/Data'

In [3]:
import torch

In [4]:
device = 'cuda' if torch.cuda.is_available() else "cpu"

In [5]:
device

'cpu'

# Data

실습에서 사용했던 영어-프랑스어와 달리 현재 사용할 데이터 셋은 영어-한국어 데이터 셋입니다. 

특히 압축을 풀어야 사용하실 수 있으니 유의하여 진행해주시길 바랍니다.

다행인 점은 용량이 그리 크지 않을겁니다!

저장 경로는 위에서 path_train_data와 path_val_data와 동일하게 맞췄습니다.

In [ ]:
!unzip -qq RNN_data/data_hw/1.Training/원천데이터/TS1.zip -d RNN_data/data_hw/1.Training/원천데이터

!unzip -qq RNN_data/data_hw/2.Validation/원천데이터/VS1.zip -d RNN_data/data_hw/2.Validation/원천데이터


## 데이터 구조 확인

In [6]:
import json

In [7]:
# 데이터 불러올 때 사용할 경로로 이후에 있을 unzip과 연동되는 내용입니다! 
path_train_data = "RNN_data/data_hw/1.Training/원천데이터/일상생활및구어체_영한_train_set.json"

path_val_data = "RNN_data/data_hw/2.Validation/원천데이터/일상생활및구어체_영한_valid_set.json"

In [8]:
# 경로는 앞서 PATH에서 선언했고 unzip에서도 경로를 지정했습니다. 만약 여기에 변화가 생기신다면 에러가 생길 수 있으니 유의하셔야 합니다.
with open(path_train_data, 'r') as f:
  raw_train_data = json.load(f)

with open(path_val_data, 'r') as f:
  raw_val_data = json.load(f)


In [9]:
# 현재 데이터의 구조가 {'data':~~~ }로 되어있는것을 알 수 있습니다.
raw_train_data.keys()

dict_keys(['data'])

In [ ]:
# 학습 데이터 개수 확인
len(raw_train_data['data'])

1200307

지금 데이터가 너무 많아서 전처리만 해도 1,2시간 걸릴 수 있습니다. 그래서 숙제에서는 매우 일부 데이터만 이용하도록 하겠습니다.

In [11]:
raw_train_data = raw_train_data['data'][:10000]
raw_val_data = raw_val_data['data'][:1000]

sample_data = raw_train_data[0]

TypeError: ignored

In [ ]:
# 샘플로 하나 확인
sample_data = raw_train_data[0]

In [12]:
# 아래의 예시를 보고 데이터를 어떻게 처리할지 잘 생각하셔야 합니다!
# 보았을 때 우리가 사용해야 하는 건 ko와 en가 되고
# 실제 데이터를 사용한다고 할 때 이 외에 필요한 데이터가 더 있을 수 있습니다.
# 모델링 과정에서는 어떻게 데이터들이 저장되어 있는지 주의해서 처리하시면 됩니다.
sample_data

{'sn': 'ECOAR1A00003',
 'data_set': '일상생활및구어체',
 'domain': '해외고객과의채팅',
 'subdomain': '숙박,음식점',
 'en_original': "I'm glad to hear that, and I hope you do consider doing business with us.",
 'en': "I'm glad to hear that, and I hope you do consider doing business with us.",
 'mt': '그 소식을 들으니 기쁩니다. 우리와 거래하는 것을 고려해 보시기 바랍니다.',
 'ko': '그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다.',
 'source_language': 'en',
 'target_language': 'ko',
 'word_count_ko': 10.0,
 'word_count_en': 15.0,
 'word_ratio': 0.667,
 'file_name': '해외고객과의채팅_숙박,음식점.xlsx',
 'source': '크라우드 소싱',
 'license': 'open',
 'style': '구어체',
 'included_unknown_words': False,
 'ner': None}

# Preprocess

세션 설명 코드에서는 알파벳 하나하나 사용했지만, 이번 과정에서는 토큰화를 진행하여 사용하는 과정을 Dataset에 바로 적용해보고자 합니다! 

토큰화는 앞서 word embedding 세션에서 사용한 것들을 그대로 이용하고자 합니다. 세부 커스터마이징은 자유롭습니다.

전처리 과정에서 중요한 점은, 앞의 구현 코드에서 여러 개의 for loop을 이용해서 구현했었지만, 실제 데이터를 사용한다면 매우 오래 걸리게 됩니다. 실제 데이터를 사용할 때엔 결국 얼마나 효율적으로 빠르게 정리할 수 있을지 고민해보시면서 접근하시길 바랍니다.

즉, 전처리에서 챙길 통찰은 얼마나 "효율적"으로 할 수 있는가? 입니다.

## Tokenizer

In [ ]:
#konlpy 설치 (mecab 제외). 3~40초 정도 소요
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

In [ ]:
# java 경로 설정
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [15]:
from konlpy.tag import Okt, Komoran

In [16]:
okt = Okt()
kom = Komoran()

In [17]:
def CustomTokenizer(sentence) :

  token = []

  stop_pos = ['JKS' , 'JKC' , 'JKG' , 'JKO' , 'JKB' , 'JKV' , 'JKQ' , 'JC' , 'SS']

  for i, j in kom.pos(sentence) :

    if j in stop_pos :

      continue
    
    token.append(i)

  return token

In [18]:
# 한국어 처리 어떻게 되고 있는지 확인
CustomTokenizer(sample_data['ko'])

['그',
 '말',
 '들',
 '으니',
 '기쁘',
 '고',
 ',',
 '저희',
 '거래',
 '하',
 '는',
 '것',
 '고려',
 '하',
 '아',
 '주',
 '시',
 '었',
 '으면',
 '하',
 'ㅂ니다',
 '.']

In [19]:
# 영어도 일단 토큰화가 되고 있으니 사용해보도록 합시다
CustomTokenizer(sample_data['en'])

['I',
 'm',
 'glad',
 'to',
 'hear',
 'that',
 ',',
 'and',
 'I',
 'hope',
 'you',
 'do',
 'consider',
 'doing',
 'business',
 'with',
 'us',
 '.']

## 본격적인 전처리 과정

전처리릏 할 때 주의할 점은, 우리가 가지고 있는 것은 train 데이터 뿐이라는 점입니다.

토큰화를 하고, token들을 수집할 때 평가데이터(혹은 테스트 데이터)에 대한 정보는 가지고 있을 수 없습니다. 그렇기 때문에 train data를 기준으로 전처리가 진행되어야 합니다.

### Tokenization

In [20]:
from tqdm import tqdm

In [26]:
print(sample_data['ko'])
print('\n')
print(sample_data['en'])

그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다.


I'm glad to hear that, and I hope you do consider doing business with us.


In [39]:
src_tmp = CustomTokenizer(sample_data['ko'])
print(src_tmp)

tar_tmp = CustomTokenizer(sample_data['en'])
print(tar_tmp)

['그', '말', '들', '으니', '기쁘', '고', ',', '저희', '거래', '하', '는', '것', '고려', '하', '아', '주', '시', '었', '으면', '하', 'ㅂ니다', '.']
['I', 'm', 'glad', 'to', 'hear', 'that', ',', 'and', 'I', 'hope', 'you', 'do', 'consider', 'doing', 'business', 'with', 'us', '.']


In [42]:
src_tmp.append('\n')
print(src_tmp)
  
tar_tmp.insert(0, '\t')
tar_tmp.append('\n')
print(tar_tmp)

['그', '말', '들', '으니', '기쁘', '고', ',', '저희', '거래', '하', '는', '것', '고려', '하', '아', '주', '시', '었', '으면', '하', 'ㅂ니다', '.', '\n']
['\t', 'I', 'm', 'glad', 'to', 'hear', 'that', ',', 'and', 'I', 'hope', 'you', 'do', 'consider', 'doing', 'business', 'with', 'us', '.', '\n']


In [75]:
src_vocab = set()
tar_vocab = set()

src_seq_tr = []
tar_seq_tr = []

# 전체 raw data에서 문장하나씩 loop 돌아가는 것
for raw_tr_dat in tqdm(raw_train_data) :

  # 문장별 tokenization
  src_tmp = CustomTokenizer(raw_tr_dat['ko'])
  tar_tmp = CustomTokenizer(raw_tr_dat['en'])

  # 입력 시퀀스에서 문장의 끝을 알리는 <EOS> 토큰 추가
  # 출력 시퀀스에서 시작과 끝을 알리는 <SOS>, <EOS> 토큰 추가
  src_tmp.append('\n')
  
  tar_tmp.insert(0, '\t')
  tar_tmp.append('\n')

  # 한국어, 영어 단어집합 구성
  for vocab in src_tmp :

     src_vocab.add(vocab)

  for vocab in tar_tmp :

     tar_vocab.add(vocab)

  # tokenization 끝난 문장 보관
  src_seq_tr.append(src_tmp)
  tar_seq_tr.append(tar_tmp)

100%|██████████| 10000/10000 [00:12<00:00, 824.90it/s]


In [48]:
print(list(src_vocab)[:5])
print(list(tar_vocab)[:5])

['신입사원', '전자 상거래', '보안과', '음료', '신체']
['sign', 'radioactive', 'circuits', 'updates', 'wanting']


In [49]:
print(src_seq_tr[:5])
print(tar_seq_tr[:5])

[['그', '말', '들', '으니', '기쁘', '고', ',', '저희', '거래', '하', '는', '것', '고려', '하', '아', '주', '시', '었', '으면', '하', 'ㅂ니다', '.', '\n'], ['확실히', '생각', '하', '고', '있', '습니다만', ',', '몇', '가지', '여쭈', '어', '보', '고', '싶', '은', '게', '있', '어요', '.', '\n'], ['오늘날', '세계', '5', '가구', '중', '1', '가구', '는', '고양', '개', '또는', '둘', '다르', 'ㄹ', '키우', '고', '있', '습니다', '.', '\n'], ['그', '들', '말', '하', '면', ',', '저희', '그', '아이', '들', '돌보', 'ㄹ', '것', '이', 'ㅂ니다', '.', '\n'], ['좋', '아요', ',', '수영', '은', '어떻', 'ㄴ가요', '?', '\n']]
[['\t', 'I', 'm', 'glad', 'to', 'hear', 'that', ',', 'and', 'I', 'hope', 'you', 'do', 'consider', 'doing', 'business', 'with', 'us', '.', '\n'], ['\t', 'I', 'm', 'definitely', 'thinking', 'about', 'it', ',', 'but', 'I', 'have', 'some', 'queries', 'to', 'ask', 'you', '.', '\n'], ['\t', 'In', 'today', 's', 'world', ',', 'one', 'in', 'every', 'five', 'families', 'has', 'either', 'a', 'cat', ',', 'dog', ',', 'or', 'both', '.', '\n'], ['\t', 'When', 'you', 'tell', 'them', ',', 'we', 'll', 'take', 'car

고민해볼 점: 과연 한국어 단어집합에 한국어만 있을까요?

지금 단계에서는 시간 상 체크하진 않지만 고민해볼 부분이기도 합니다.

### token - index 정리

In [51]:
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print('source 문장의 char 집합 :',src_vocab_size)
print('target 문장의 char 집합 :',tar_vocab_size)

source 문장의 char 집합 : 5991
target 문장의 char 집합 : 7031


In [76]:
# 최종 정수 인코딩 하기 전 보기 편하기 위해 sort
src_vocab = sorted(list(src_vocab))
tar_vocab = sorted(list(tar_vocab))

print(src_vocab[:30])
print(tar_vocab[:30])

['\n', '!', '$', '%', ',', '-', '-4', '-5', '.', '.com', '0', '000', '0000000', '006', '1', '10', '100', '1000', '10년', '10년 후', '10대', '10분', '10월', '10월 15일', '10월 20일', '10월 27일', '11', '1100', '11월', '11월 13일']
['\t', '\n', '!', '$', '%', ',', '-', '.', '.com', '/', '0', '00', '000', '0000000', '006', '1', '10', '100', '1000', '11', '1100', '12', '120', '123', '1234', '123456', '1234567', '12345678', '123456789', '123456789012']


In [53]:
# token - idx dictionary
# 여기서도 동일하게 i+1로 해줘야 padding에 사용할 token을 0으로 사용할 수 있습니다!
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{'\n': 1, '!': 2, '$': 3, '%': 4, ',': 5, '-': 6, '-4': 7, '-5': 8, '.': 9, '.com': 10, '0': 11, '000': 12, '0000000': 13, '006': 14, '1': 15, '10': 16, '100': 17, '1000': 18, '10년': 19, '10년 후': 20, '10대': 21, '10분': 22, '10월': 23, '10월 15일': 24, '10월 20일': 25, '10월 27일': 26, '11': 27, '1100': 28, '11월': 29, '11월 13일': 30, '11월 15일': 31, '11월 1일': 32, '11월 29일': 33, '11월 5일': 34, '11월 7일': 35, '11월 8일': 36, '11월 9일': 37, '12': 38, '120': 39, '1200': 40, '123': 41, '1234': 42, '123456': 43, '1234567': 44, '12345678': 45, '123456789': 46, '123546789': 47, '125': 48, '128': 49, '12월': 50, '12월 1일': 51, '12월 20일': 52, '12월 3일': 53, '13': 54, '135': 55, '14': 56, '140': 57, '142': 58, '144': 59, '15': 60, '150': 61, '15년': 62, '16': 63, '1600': 64, '17': 65, '18': 66, '19': 67, '1952': 68, '1952년': 69, '1년': 70, '1세기': 71, '1월': 72, '1월 8일': 73, '1월 9일': 74, '2': 75, '20': 76, '200': 77, '2000': 78, '2002': 79, '200569978005': 80, '2019': 81, '2021': 82, '2022': 83, '2070': 84, '20대': 85, 

### 정수 인코딩

In [56]:
src_seq_tr[0]

['그',
 '말',
 '들',
 '으니',
 '기쁘',
 '고',
 ',',
 '저희',
 '거래',
 '하',
 '는',
 '것',
 '고려',
 '하',
 '아',
 '주',
 '시',
 '었',
 '으면',
 '하',
 'ㅂ니다',
 '.',
 '\n']

In [134]:
# 현재 2가지 언어에 대해서 각각 단어집합을 만든 상태입니다. 즉 사용하는 언어들마다 단어집합도 다르고, index도 다릅니다.
# 그래서 입력 시퀀스 출력 시퀀스별로 token에 정수형의 숫자들을 할당해놓은 것이 바로 src_to_index, tar_to_index입니다.

# 아래의 코드에서는 이를 이용해서 문자 --> 숫자로 바꿔주는 과정입니다.

# 입력 시퀀스를 먼저 처리하겠습니다.
# 입력 시퀀스는 encoder에 들어가는 값이기에 encoder_input이라고 해보겠습니다.
encoder_input = []

# 1개의 문장
for sentence in src_seq_tr :

  encoded_line = []
  
  # 각 줄에서 1개의 알파벳별로 처리. 앞서서 동일한 방식으로 tokenization을 했기 때문!
  for word in sentence :
    
    # 각 알파벳을 정수로 바꾸어야합니다. 왜 하필 정수형이냐에 대해선 이후에 있을 Embedding과 관련 있습니다.
    # 우선 각 알파벳을 숫자로 바꿔줍니다.
    encoded_line.append(src_to_index[word])
  
  # 위에서 사용한 index도 정수형이긴한데, 결국 torch.Tensor를 사용해야 하고, Tensor에서 정수형에 해당하는 type이 LongTensor이기 때문에
  # .long()을 이용하여 정수형으로 만들어 줍니다.
  encoder_input.append(torch.Tensor(encoded_line).long())

print('source 문장의 정수 인코딩 :\n',encoder_input[:5], '\n')

source 문장의 정수 인코딩 :
 [tensor([ 873, 1827, 1577, 3981,  992,  649,    5, 4422,  506, 5661, 1244,  562,
         662, 5661, 3285, 4678, 3132, 3545, 3990, 5661,  341,    9,    1]), tensor([5907, 2735, 5661,  649, 4263, 3129,    5, 1942,  400, 3593, 3473, 2308,
         649, 3269, 3994,  563, 4263, 3499,    9,    1]), tensor([3693, 2844,  128,  353, 4730,   15,  353, 1244,  676,  472, 1636, 1553,
        1284,  328, 5268,  649, 4263, 3128,    9,    1]), tensor([ 873, 1577, 1827, 5661, 1917,    5, 4422,  873, 3325, 1577, 1510,  328,
         562, 4049,  341,    9,    1]), tensor([4668, 3321,    5, 2997, 3994, 3485,  315,  175,    1])] 



In [135]:
# decoder에 사용할 입력값도 숫자로 바꾸는 과정이 들어갑니다.
# 또한 Decoder의 경우 입력 데이터와 출력 데이터의 형태가 1 step씩 차이가 나는 구조입니다!
# 이를 유의하여 보셔야 합니다.


# 현재 데이터는 "<SOS> 문장 <EOS>" 구조입니다.
# 아래의 코드는 Decoder의 입력 데이터를 만드는 코드이기에 EOS가 제외되어야합니다.
# 그리고 EOS는 문장(list)에서 맨 마.지.막.에 있습니다!

decoder_input = []

# 여기서 line은 문자열에 해당합니다. 
for sentence in tar_seq_tr :

  encoded_line = []
  
  for word in sentence :

    encoded_line.append(tar_to_index[word])
  
  # 현재 데이터 상 문장 맨 마지막에는 <EOS>가 들어가 있는데, decoder를 학습할 때 <EOS>를 넣지 않으니 마지막의 값을 pop하여 제외합니다.
  encoded_line.pop(-1)

  # 여기서 .long()은 정수 인코딩된 문장들의 type을 정하는 것으로, 이후에 Embedding layer를 사용할 때 해당 타입을 사용해야하기 때문에 사용한 것입니다.
  decoder_input.append(torch.Tensor(encoded_line).long())  
  
print('target 문장의 정수 인코딩 :\n',decoder_input[:5],'\n')

target 문장의 정수 인코딩 :
 [tensor([   1,  448, 4154, 3359, 6471, 3509, 6400,    6, 1137,  448, 3576, 7018,
        2588, 2101, 2602, 1640, 6952, 6707,    8]), tensor([   1,  448, 4154, 2398, 6425,  907, 3891,    6, 1645,  448, 3498, 5957,
        5208, 6471, 1248, 7018,    8]), tensor([   1,  460, 6475, 5609, 6977,    6, 4593, 3688, 2896, 3149, 3042, 3493,
        2734,  903, 1742,    6, 2600,    6, 4629, 1555,    8]), tensor([   1,  872, 7018, 6354, 6405,    6, 6874, 4089, 6292, 1716, 4566, 6403,
        1837,    8]), tensor([   1,  613,    6, 3603,  907, 3206, 6267,  124])] 



In [136]:
# decoder에서 출력할 값도 숫자로 바꿔주는 과정입니다.
# 앞선 과정과 비슷하게 진행되고 차이점으로는 1step씩 차이가 나야 합니다.
# 이를 유의하여 보시길 바랍니다.

decoder_target = []

for sentence in tar_seq_tr :

  # 여기서 timestep은 시퀀스 상 몇번째 값인지 알려주는 값입니다.
  timestep = 0
  encoded_line = []
  
  for word in sentence :

    # Decoder가 출력하는 값에서 문장의 시작을 알리는 값(SOS)가 들어가면 안됩니다.
    # 반면 현재 데이터는 SOS로 시작하는 형태입니다. 
    # 그래서 시퀀스에서 0번째를 제외한 경우만 추가를 하게 된다면 적절히 제외될 것입니다. 
    if timestep > 0 :

      encoded_line.append(tar_to_index[word])
    
    timestep = timestep + 1
  
  decoder_target.append(torch.Tensor(encoded_line).long())

print('target 문장 레이블의 정수 인코딩 :\n',decoder_target[:5],'\n')

target 문장 레이블의 정수 인코딩 :
 [tensor([ 448, 4154, 3359, 6471, 3509, 6400,    6, 1137,  448, 3576, 7018, 2588,
        2101, 2602, 1640, 6952, 6707,    8,    2]), tensor([ 448, 4154, 2398, 6425,  907, 3891,    6, 1645,  448, 3498, 5957, 5208,
        6471, 1248, 7018,    8,    2]), tensor([ 460, 6475, 5609, 6977,    6, 4593, 3688, 2896, 3149, 3042, 3493, 2734,
         903, 1742,    6, 2600,    6, 4629, 1555,    8,    2]), tensor([ 872, 7018, 6354, 6405,    6, 6874, 4089, 6292, 1716, 4566, 6403, 1837,
           8,    2]), tensor([ 613,    6, 3603,  907, 3206, 6267,  124,    2])] 



In [60]:
from torch.nn.utils.rnn import pad_sequence

### Padding

In [61]:
# 현재 길이가 제각각인 정수화된 문장이 있습니다. padding을 통해 길이를 맞춰봅시다
# 주의사항: 여기에 사용된 값을은 정수일까요?
batch_first = True

encoder_input = pad_sequence(encoder_input, batch_first = batch_first).long()
decoder_input = pad_sequence(decoder_input, batch_first = batch_first).long()
decoder_target = pad_sequence(decoder_target, batch_first = batch_first).long()

## Train data

tuple type으로,  (encoder_input, decoder_input, decoder_target)의 순서를 가지게 됩니다. 이는 tuple assignment를 이용하려고 하는 것이니 자유롭게 사용하셔도 좋습니다.

In [62]:
train_data = tuple([encoder_input, decoder_input, decoder_target])

## Eval data

지금까지 진행한 것은 학습 데이터에 대한 전처리입니다.

그런데 말입니다, 과연 평가 데이터는 학습데이터랑 동일한 형태일까요?

사용하는 단어도 동일할까요?

게다가 실제 데이터라면 문장 속에 이메일이나, 전화번호등 다른 것들로 대체해야할 게 있지 않을까요?

참으로 고민할 게 많습니다. 이 모든 것들을 고려하면 좋겠지만, 현재 과제에서는 학습 데이터의 단어 집합에는 없는데 평가 데이터에서 등장하는 단어들은 어떻게 처리할지 한번 고민해봅시다.

**핵심 TODO**:만약에 본 적이 없는 token이 나온다면 어떻게 될까요?

--> Out of Vocabulary !  =지금까지 단어 집합에 없던 아이들!<br>

이를 한번에 처리해주는 토큰으로 \<UNK\>(=unknown token) 을 사용

EX)

train data : "나는 밥을 먹는다."<br>
\>> ["나", "밥", "먹다"]<br>
\>> [\<SOS\>, 나, 밥, 먹다, \<EOS\>] <BR>
\>> {\<SOS\>:0, \<EOS\>:1, \<UNK\>: 2, 나:3, 밥:4, 먹다:5}  <bR>
\>> [0, 3, 4, 5, 1] <BR>

val data: "나는 고기를 먹는다"<br>
\>> ['나', '고기' '먹다']<BR>
\>> [\<SOS\>, 나, \<UNK\>, 먹다, \<EOS\>] <BR>
\>> [0, 3, 2, 5, 1] <BR>

In [107]:
src_vocab_ex = ['나는' , '오늘' , '저녁으로' , '피자를' , '먹었다']ㅁ
src_tmp_ex = ['나는' , '오늘' , '저녁으로' , '치킨을' , '먹었다']

list(filter(lambda x : x not in src_vocab_ex , src_tmp_ex))

['치킨을']

In [113]:
src_oov_ex = ['저희']

token_list = CustomTokenizer(sample_data['ko'])

for index, value in enumerate(token_list):
    
    if value in src_oov_ex :
      
      token_list[index] = 'UNK'

token_list

['그',
 '말',
 '들',
 '으니',
 '기쁘',
 '고',
 ',',
 'UNK',
 '거래',
 '하',
 '는',
 '것',
 '고려',
 '하',
 '아',
 '주',
 '시',
 '었',
 '으면',
 '하',
 'ㅂ니다',
 '.']

In [128]:
src_seq_val = []
tar_seq_val = []

for raw_val_dat in tqdm(raw_val_data):
  # 문장별 tokenization
  src_tmp = CustomTokenizer(raw_val_dat['ko'])
  tar_tmp = CustomTokenizer(raw_val_dat['en'])

  # 입력 시퀀스에서 문장의 끝을 알리는 <EOS> 토큰 추가
  # 출력 시퀀스에서 시작과 끝을 알리는 <SOS>, <EOS> 토큰 추가
  src_tmp.append('\n')
  
  tar_tmp.insert(0, '\t')
  tar_tmp.append('\n')

  # 처음보는 단어들은 일단 모르는 단어(OOV)로 표시해놓기  
  # 해당 과정에서는 지금 얻은 toekn들(src_tmp와 tar_tmp)들 중에서 학습 데이터의 단어 집합(src_vocab과 tar_vocab)에 없는 단어들을 찾아야 합니다.
  # 즉 위의 예시에서 "고기"에 해당하는 token을 찾아야 하고, 이를 list로 저장해봅시다.
  src_oov = list(filter(lambda x : x not in src_vocab , src_tmp))
  tar_oov = list(filter(lambda x : x not in tar_vocab , tar_tmp))

  # OOV들은 <UNK>로 바꾸는 과정입니다.
  # 지금 문장을 tokenization되어 있고, 앞서 구한 모르는 단어(OOV)들의 리스트(src_oov와 tar_oov)가 있는 상황에서
  # 해당 리스트에 속한 토큰들은 <UNK>토큰으로 바꿔주는 것을 구현하는 파트입니다.
  # 위의 예시에서 본다면 [<SOS>, 나, 고기, 먹다, <EOS>]를 [<SOS>, 나, <UNK>, 먹다, <EOS>]로 만들어주는 과정입니다.
  for index, value in enumerate(src_tmp):
    
    if value in src_oov :
      
      src_tmp[index] = '<UNK>'

  for index, value in enumerate(tar_tmp):
    
    if value in tar_oov :
      
      tar_tmp[index] = '<UNK>'
  
  # tokenization 끝난 문장 보관
  src_seq_val.append(src_tmp)
  tar_seq_val.append(tar_tmp)

100%|██████████| 1000/1000 [00:05<00:00, 172.57it/s]


In [129]:
tar_seq_val[5]

['\t',
 '<UNK>',
 '<UNK>',
 ',',
 '<UNK>',
 ',',
 'and',
 'even',
 '<UNK>',
 'skin',
 '.',
 '\n']

위의 샘플을 보면 생각보다 너무 많은 단어들이 UNK로 바뀌었습니다. 이게 과연 성능에 어떤 영향을 끼칠까요? 이를 방지할 수 있는 대책은 무엇일까요?

정답은 없습니다. 자유롭게 고민해보셔도 좋을 것 같습니다.

### UNK 토큰 추가 및 단어집합, token-index 업데이트

In [ ]:
# UNK가 추가되면서 vocab과 index도 업데이트를 해야합니다.
# 즉 지금 방식으로 처리한다면, 지금 단어 집합에는 <UNK>라는 토큰이 없습니다.
# 그래서 UNK 토큰에도 정수를 할당해줘야 합니다.
# 지금 0은 padding에서 사용하고 있으니 아래의 과정은 마지막에 <UNK>를 추가해보는 것입니다.

In [118]:
src_to_index

{'\n': 1,
 '!': 2,
 '$': 3,
 '%': 4,
 ',': 5,
 '-': 6,
 '-4': 7,
 '-5': 8,
 '.': 9,
 '.com': 10,
 '0': 11,
 '000': 12,
 '0000000': 13,
 '006': 14,
 '1': 15,
 '10': 16,
 '100': 17,
 '1000': 18,
 '10년': 19,
 '10년 후': 20,
 '10대': 21,
 '10분': 22,
 '10월': 23,
 '10월 15일': 24,
 '10월 20일': 25,
 '10월 27일': 26,
 '11': 27,
 '1100': 28,
 '11월': 29,
 '11월 13일': 30,
 '11월 15일': 31,
 '11월 1일': 32,
 '11월 29일': 33,
 '11월 5일': 34,
 '11월 7일': 35,
 '11월 8일': 36,
 '11월 9일': 37,
 '12': 38,
 '120': 39,
 '1200': 40,
 '123': 41,
 '1234': 42,
 '123456': 43,
 '1234567': 44,
 '12345678': 45,
 '123456789': 46,
 '123546789': 47,
 '125': 48,
 '128': 49,
 '12월': 50,
 '12월 1일': 51,
 '12월 20일': 52,
 '12월 3일': 53,
 '13': 54,
 '135': 55,
 '14': 56,
 '140': 57,
 '142': 58,
 '144': 59,
 '15': 60,
 '150': 61,
 '15년': 62,
 '16': 63,
 '1600': 64,
 '17': 65,
 '18': 66,
 '19': 67,
 '1952': 68,
 '1952년': 69,
 '1년': 70,
 '1세기': 71,
 '1월': 72,
 '1월 8일': 73,
 '1월 9일': 74,
 '2': 75,
 '20': 76,
 '200': 77,
 '2000': 78,
 '2002': 79,
 

In [116]:
# 입력 시퀀스의 마지막 토큰과 정수인코딩값 확인
list(src_to_index.items())[-1]

('힘쓰', 5990)

In [117]:
# 현재 입력 시퀀스에 사용된 token들의 개수 확인
len(src_vocab)

5990

In [130]:
# 우리는 마지막 토큰 뒤에 UNK 토큰을 추가해봅시다
# 이 때 입력 시퀀스(src_to_index)와 출력시퀀스(tar_to_index)에 모두 사용해줘야 합니다.
src_to_index["<UNK>"] = len(src_vocab) + 1
tar_to_index["<UNK>"] = len(tar_vocab) + 1

In [121]:
# 입력시퀀스의 단어집합 확인
print(list(src_to_index.items())[:5])
print(list(src_to_index.items())[-1:-5:-1])

[('\n', 1), ('!', 2), ('$', 3), ('%', 4), (',', 5)]
[('<UNK>', 5991), ('힘쓰', 5990), ('힘들', 5989), ('힘', 5988)]


In [120]:
# 출략시퀀스의 단어집합 확인
print(list(tar_to_index.items())[:5])
print(list(tar_to_index.items())[-1:-5:-1])

[('\t', 1), ('\n', 2), ('!', 3), ('$', 4), ('%', 5)]
[('<UNK>', 7031), ('é', 7030), ('zoom', 7029), ('zones', 7028)]


### 정수 인코딩

In [140]:
# 현재 2가지 언어에 대해서 각각 단어집합을 만든 상태입니다. 즉 사용하는 언어들마다 단어집합도 다르고, index도 다릅니다.
# 그래서 입력 시퀀스 출력 시퀀스별로 token에 정수형의 숫자들을 할당해놓은 것이 바로 src_to_index, tar_to_index입니다.

# 아래의 코드에서는 이를 이용해서 문자 --> 숫자로 바꿔주는 과정입니다.

# 입력 시퀀스를 먼저 처리하겠습니다.
# 입력 시퀀스는 encoder에 들어가는 값이기에 encoder_input이라고 해보겠습니다.
encoder_input_val = []

# 1개의 문장
for sentence in src_seq_val :

  encoded_line = []
  
  # 각 줄에서 1개의 알파벳별로 처리. 앞서서 동일한 방식으로 tokenization을 했기 때문!
  for word in sentence :
    
    # 각 알파벳을 정수로 바꾸어야합니다. 왜 하필 정수형이냐에 대해선 이후에 있을 Embedding과 관련 있습니다.
    # 우선 각 알파벳을 숫자로 바꿔줍니다.
    encoded_line.append(src_to_index[word])
  
  # 위에서 사용한 index도 정수형이긴한데, 결국 torch.Tensor를 사용해야 하고, Tensor에서 정수형에 해당하는 type이 LongTensor이기 때문에
  # .long()을 이용하여 정수형으로 만들어 줍니다.
  encoder_input_val.append(torch.Tensor(encoded_line).long())

print('source 문장의 정수 인코딩 :\n',encoder_input_val[:5], '\n')

source 문장의 정수 인코딩 :
 [tensor([5991, 1950,    5,   56, 4196, 2222, 5991,    5, 4016, 5991, 4594, 1596,
        2982, 2102, 2997, 5398, 1674, 5274, 2194, 3061, 3378,  339,  977, 4263,
        3128,    9,    1]), tensor([5326, 1244,  100, 2848,  466, 3938, 4797, 2200, 5991, 4433, 1545, 3545,
        3128,    9,    1]), tensor([4605, 2034, 1244, 4049, 1909, 4678, 3132, 1917,  435, 5661,  578, 3128,
           9,    1]), tensor([3355,    5,  193, 5933,  181, 4049,  341,    9,    1]), tensor([4406, 1244, 5991, 1674, 1593, 5991, 5991, 3085, 2793, 2867, 4605, 2883,
        5661, 1704, 5661,  341,    9,    1])] 



In [141]:
# decoder에 사용할 입력값도 숫자로 바꾸는 과정이 들어갑니다.
# 또한 Decoder의 경우 입력 데이터와 출력 데이터의 형태가 1 step씩 차이가 나는 구조입니다!
# 이를 유의하여 보셔야 합니다.


# 현재 데이터는 "<SOS> 문장 <EOS>" 구조입니다.
# 아래의 코드는 Decoder의 입력 데이터를 만드는 코드이기에 EOS가 제외되어야합니다.
# 그리고 EOS는 문장(list)에서 맨 마.지.막.에 있습니다!

decoder_input_val = []

# 여기서 line은 문자열에 해당합니다. 
for sentence in tar_seq_val :

  encoded_line = []
  
  for word in sentence :

    encoded_line.append(tar_to_index[word])
  
  # 현재 데이터 상 문장 맨 마지막에는 <EOS>가 들어가 있는데, decoder를 학습할 때 <EOS>를 넣지 않으니 마지막의 값을 pop하여 제외합니다.
  encoded_line.pop(-1)

  # 여기서 .long()은 정수 인코딩된 문장들의 type을 정하는 것으로, 이후에 Embedding layer를 사용할 때 해당 타입을 사용해야하기 때문에 사용한 것입니다.
  decoder_input_val.append(torch.Tensor(encoded_line).long())  
  
print('target 문장의 정수 인코딩 :\n',decoder_input_val[:5],'\n')

target 문장의 정수 인코딩 :
 [tensor([   1, 7031, 3693, 3513, 5245, 4385,    6,   36, 2342, 1406, 4046,    6,
        4433, 4908, 2158, 2558, 5901, 1137, 7031, 6520, 6869, 5918, 7031,    8]), tensor([   1,  790, 6503, 5687, 3493,   68, 5254, 3330, 7031, 3362, 1137, 7031,
        3134, 7031, 1191,    8]), tensor([   1,  383, 5081, 3775,    6, 4917, 5747, 6707, 1130, 2759,    8]), tensor([   1,  435,    6,  193, 1994,  127,    8]), tensor([   1,  448, 6990, 4055, 6471, 3854, 6401, 7031, 7031, 7031, 7031, 7031,
        7031,    8])] 



In [142]:
# decoder에서 출력할 값도 숫자로 바꿔주는 과정입니다.
# 앞선 과정과 비슷하게 진행되고 차이점으로는 1step씩 차이가 나야 합니다.
# 이를 유의하여 보시길 바랍니다.

decoder_target_val = []

for sentence in tar_seq_val :

  # 여기서 timestep은 시퀀스 상 몇번째 값인지 알려주는 값입니다.
  timestep = 0
  encoded_line = []
  
  for word in sentence :

    # Decoder가 출력하는 값에서 문장의 시작을 알리는 값(SOS)가 들어가면 안됩니다.
    # 반면 현재 데이터는 SOS로 시작하는 형태입니다. 
    # 그래서 시퀀스에서 0번째를 제외한 경우만 추가를 하게 된다면 적절히 제외될 것입니다. 
    if timestep > 0 :

      encoded_line.append(tar_to_index[word])
    
    timestep = timestep + 1
  
  decoder_target_val.append(torch.Tensor(encoded_line).long())

print('target 문장 레이블의 정수 인코딩 :\n',decoder_target_val[:5],'\n')

target 문장 레이블의 정수 인코딩 :
 [tensor([7031, 3693, 3513, 5245, 4385,    6,   36, 2342, 1406, 4046,    6, 4433,
        4908, 2158, 2558, 5901, 1137, 7031, 6520, 6869, 5918, 7031,    8,    2]), tensor([ 790, 6503, 5687, 3493,   68, 5254, 3330, 7031, 3362, 1137, 7031, 3134,
        7031, 1191,    8,    2]), tensor([ 383, 5081, 3775,    6, 4917, 5747, 6707, 1130, 2759,    8,    2]), tensor([ 435,    6,  193, 1994,  127,    8,    2]), tensor([ 448, 6990, 4055, 6471, 3854, 6401, 7031, 7031, 7031, 7031, 7031, 7031,
           8,    2])] 



### Padding

In [ ]:
# 주의사항: 여기에 사용된 값을은 정수일까요?
encoder_input_val = ~~~
decoder_input_val = ~~~
decoder_target_val = ~~~

In [143]:
val_data = tuple([encoder_input_val, decoder_input_val, decoder_target_val])

# Dataset

실습코드에서 사용한 것과 동일한 구조입니다! 

사실 앞서 처리가 진행되었던 모든 과정은 Dataset에서 __get_item__ 함수 내에서 처리해서 값을 리턴할 수도 있습니다.

다만 설명을 위해 이번 과제에서는 과정을 단계별로 설명 드렸습니다. 추후에 진행하실 때엔 __get_item__에서 구현해보셔도 좋을 것 같아요!

그리고 일부 단계의 경우 임의로 처리된 것들이 있습니다.(tokenization부터 stopwords 등등) 가령 이메일, 전화번호와 같은 단어들은 \<UNK\>처럼 새로운 토큰을 이용해서 처리할 수도 있으며 목적에 따라서 다른 tokenziation을 사용할 수 있습니다.

**(참고)**

 __get_item__말고 사전에 전처리를 다한 상태로 사용할 경우 사실 사용할 모든 데이터를 메모리에 올려놓고 사용하는 것과 같습니다.(변수에 할당되어 있으니)

그런데 우리가 데이터가 매매매매매우 많아진다면, 메모리가 부족해질 수도 있지 않을까요? 그렇기 때문에 때에 따라 data를 필요할 때 전처리를 하고 반환하기도 합니다. 여기서 data가 필요할때란 결국 __get_item__을 호출할 때가 될 것입니다.

정리하자면, 꼭 모든 데이터를 한번에 다 전처리해놓도 준비해놓지 않고도, 필요할 때 마다(get_item이 호출될 때마다) 전처리해서 결과를 전달하는 방법도 있습니다.

다만 해당 과제에서는 Seq2Seq에 익숙해지는 것을 목표로 하기에 이러한 점만 알고 가셔도 충분하기도 하고, 오히려 사전에 다 처리해놓는 현재 방식이 더 효율적일 수도 있습니다. 그렇기 때문에 이 점만 알고 가셔도 좋을 것 같습니다!

In [144]:
from torch.utils.data import Dataset, DataLoader

In [145]:
class textDataset(Dataset) :

  def __init__(self, data, batch_first = True) :

    super(textDataset, self).__init__()
    
    # 앞서 tuple형태로 train과 val을 정리했었습니다. 
    # 그것들을 다시 encoder의 input(enc_inp), decoder의 input(dec_inp), decoder의 output(dec_out)로 정리하는 코드입니다.
    # tuple asgginment를 활용한 것으로 이렇게 하면 각 변수에 순서대로 할당됩니다.
    enc_inp, dec_inp, dec_out = data

    # 내부에서 사용할 변수들 정리하는 것으로, 모든 데이터가 torch.LongTensor로 정리된 값들이 저장됩니다.
    self.enc_inp = enc_inp
    self.batch_first = batch_first
    self.dec_inp = dec_inp
    self.dec_out = dec_out

  def __getitem__(self, idx) :

    # 지금 전처리가 끝난 상태로 attribute에 저장되었기 때문에 단순히 indexing만 해도 됩니다.
     return self.enc_inp[idx], self.dec_inp[idx],self.dec_out[idx]

  def __len__(self) :

    # 전체 개수를 반환해야 하는데, 이 때 batch_first에 따라서 리턴하는 것이 달라집니다.
    # batch_first=True 인 경우 (batch_size, seq_len)이니 size에서 첫번째 값을 가져와야 하기 때문에 .size(0)이 되어야 합니다.
    # 그런데 우리가 가진 것은 True 값이고, int(True) = 1이니 int(~True) = 0임을 이용해서 값을 반환하게 됩니다.
    
    return self.enc_inp.size(int(~self.batch_first))

In [146]:
train_dataset = textDataset(train_data, batch_first = batch_first)
val_dataset = textDataset(val_data, batch_first = batch_first)

In [147]:
val_dataset[0]

(tensor([5991, 1950,    5,   56, 4196, 2222, 5991,    5, 4016, 5991, 4594, 1596,
         2982, 2102, 2997, 5398, 1674, 5274, 2194, 3061, 3378,  339,  977, 4263,
         3128,    9,    1]),
 tensor([   1, 7031, 3693, 3513, 5245, 4385,    6,   36, 2342, 1406, 4046,    6,
         4433, 4908, 2158, 2558, 5901, 1137, 7031, 6520, 6869, 5918, 7031,    8]),
 tensor([7031, 3693, 3513, 5245, 4385,    6,   36, 2342, 1406, 4046,    6, 4433,
         4908, 2158, 2558, 5901, 1137, 7031, 6520, 6869, 5918, 7031,    8,    2]))

# Model

In [148]:
import torch.nn as nn
import torch.functional as F

## Encoder

본 모델에서는 LSTM를 기본 단위로 가지는 Seq2Seq 모델을 구성할 것

참고 [레퍼런스](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

In [149]:
class Encoder(nn.Module) :

  def __init__(self, in_size, hid_size, tok_size, bat_first = True, num_lay = 1, bidirect = False):
    super(Encoder, self).__init__()
    
    # 여기서 hidden_size는 LSTM의 hidden_size로 embedding vector를 몇차원의 벡터로 transform할 것인지 말합니다!
    self.hidden_size = hid_size
   
    # 앞서 설명한 내용대로 전체 사용하는 token의 수를 embedding_num에 해당하고, 이를 몇차원의 embedding dim으로 표현할 것인지 말합니다.
    # 이 때 LSTM의 관점에서 embedding dim은 input_size에 해당해서 개념적으로 embedding_dim = in_size입니다.
    self.embedding = nn.Embedding(tok_size, in_size)

    # layer를 구성할 때 LSTM 내부의 layer의 개수도 조정할 수 있습니다.
    self.num_layers = num_lay

    # 이것은 bidirectional 방식을 사용할 것인지를 얘기하는데,
    # 이는 현재 과정에서 고려하지 않는 값이지만 LSTM 레이어를 설정할 때 필요한 값이라 기록하긴 했습니다.
    self.num_directional = 2 if bidirect else 1

    self.batch_first = bat_first
    self.lstm = nn.LSTM(in_size, 
                        hid_size, 
                        num_layers = num_lay,
                        batch_first = bat_first,
                        bidirectional = bidirect)

  def forward(self, x, hidden):
    """
    현재 전체 전처리부터 모델링 과정을 batch_first=True환경에서 진행하고 있으니 이를 유의하시길 바랍니다.
    batch_first=True이기에 입력값(x)의 차원은 다음과 같고, 
    hidden_state에 해당하는 값도 batch_size가 먼저 나오게 됩니다.
    x: (batch_size, seq_len)
    hidden : (num_directional*num_layers, batch_size, hidden_size)
    
    개념적으로 보면 우리가 전처리한 시퀀스(seq)는
    seq = [정수인코딩1, 정수인코딩2, ..]에 해당합니다.
    여기서 중요한 것은 들어오는 값이 정수인코딩된 값입니다!!
    
    이런 seq들이 모여서 만들어진 것이 
    x = [seq1, seq2, ...]에 해당합니다.

    그래서 len(x) = batch_size가 되는 것입니다!    
    """

    # 여기서 중요한 점은 입력값으로 정수인코딩된 값이 들어오는 것이고, embedidng layer를 거치면서 
    # 정수인코딩값이 vector로 바뀌게 됩니다!!
    # 그런데 이상하지 않나요? 위에서 one-hot된 벡터를 입력해서 embedding vector로 바꿔주는데
    # 정수인코딩 값만 넣다니?!
    # 이는 사실 torch의 Embedding layer가 그런 역할까지 포함하고 있는 것이고, 
    # Embedding layer를 사용할 땐 "정수"인코딩된 값을 넣어주면 대응하는 embedding vector를 반환해줍니다.
    # 여기서 "정수"인코딩 값을 사용해야 하기 때문에 앞에서 .long()과정이 들어간 것입니다.
    # 안그럼 에러뜰거에요
    # emb : (batch_size, seq_len, input_size)
    emb = self.embedding(x)

    # LSTM에서 하는 연산은 결국 Linear와 크게 다르지 않습니다.
    # input_size의 임베딩 벡터를 hidden_size의 벡터로 바꾸게 되니 output의 차원은
    # out : (batch_size, seq_len, hidden_size)

    # 이와 달리 hidden_state는 그 구조가 조금 다릅니다.
    # 여기서 num_layer가 사용되는 것은 LSTM에서 내부의 layer가 많아질수록 그 만큼 hidden state가 더 나와야 할 것입니다.
    # 위에 LSTM layer에서 인용한 사진이 잘 설명하는데 내부에서 layer가 많아질수록 hidden state도 많아지게 될 것입니다.
    # 그리고 최종 반환하는 hidden_state는 앞의 시퀀스의 정보를 다 포함하는 벡터가 나온 것이고,
    # 그 모든 정보를 가지고 있는 벡터의 차원이 바로 hidden_size입니다.
    # 즉 layer마다 batch 만큼의 context vector가 담겨져 있다! 생각할 수 있을거에요
    # hidden : (num_layers, batch_size, hidden_size)
    out, hidden = self.lstm(emb, hidden)
    
    return out, hidden

## Decoder

decoder의 hidden이 아닌 out으로 예측하고, 그 값 하나를 이용해서 다음 값 계산에 사용하기 때문에 seq_len =1이 된다.

In [150]:
class Decoder(nn.Module) :

  def __init__(self, in_size, hid_size, tok_size, bat_first=True, num_lay=1, bidirect=False):
    super(Decoder, self).__init__()

    # 대부분의 방식은 사실 Encoder와 비슷하게 흘러갑니다.

    # LSTM이 embedding을 몇차원의 벡터로 변환할지 정하는 값입니다.
    self.hidden_size = hid_size

    # 여기서도 번역해야하는 언어의 token을 embedding 해야하고 이  때 embedding vector의 차원이 결국 LSTM입장에서의 Input size가 됩니다.
    self.embedding = nn.Embedding(tok_size, in_size)

    self.num_layers = num_lay
    self.num_directional = 2 if bidirect else 1
    self.batch_first = bat_first
    self.lstm = nn.LSTM(in_size, 
                        hid_size, 
                        num_layers = num_lay,
                        batch_first = bat_first,
                        bidirectional=bidirect)
    
    self.relu =  nn.ReLU()

    # 여기서 Encoder와 차이가 생기는데,
    # Decoder는 입력된 단어와 context를 보고 다음 단어를 "예측"해야합니다.
    # 그렇기 때문에 hidden_size의 벡터를 token의 개수만큼 다시 mapping해주어야
    # 각 token별 점수로 만들어줘야 최종 예측을 할 수 있습니다.
    # 그래서 최종 값을 반환할 때에는 총 토큰 수만큼으로 만들어 줘야 토큰별 확률을 계산하게 됩니다.
    self.fc1 = nn.Linear(hidden_size, tok_size)
    
  def forward(self, x, hidden):
    """
    x: (batch_size, seq_len=1)
    hidden: (num_directional*num_layers, batch_size, hidden_size)
    """
    
    # 여기서 조금 trivial한 점을 고려해야 합니다.
    # Embedding layer를 거치고 나왔을 때 차원은 아래와 같아야 하고 이를 위해서 x[:, 0]과 .unsqueeze(1)을 한 것입니다. 이것은 정해진 답은 없고, 필요하신 대로 이용하셔도 좋습니다!
    # emb: (batch_size, seq_len=1, input_size)   
    # x[:, 0] : (batch_size, )
    # self.embedding(x[:, 0]) : (batch_size, input_size)
    # self.embedding(x[:, 0]).unsqueeze(1) : (batch_size, 1, input_size)
    emb = self.embedding(x[:, 0]).unsqueeze(1)
    # 이거 말고도 동일한 차원을 가지도록 처리하려면 어떻게 해야할까요?
    # emb = self.embedding(~~)~~

    # relu를 취한 것은 별다른 이유가 없이 참고한 레퍼런스와 동일한 구조로 해본 것입니다.
    out = self.relu(emb)

    # LSTM을 사용한다는 점에서 Encoder와 동일하게 output과 hidden_state는 동일한 구조로 반환하게 됩니다.
    # 다만 seq_len은 decoder에서 1이 된다는 점입니다!
    # out : (batch_size, seq_len=1, hidden_size)
    # hidden: (num_layers, batch_size, hidden_size)
    out, hidden = self.lstm(out, hidden)

    # 기존의 output에서 seq_len = 1이니 squeeze하게 된다면 (batch_size, hidden_size)가 되고
    ## out.squeeze(1) : (batch_size, hidden_size)
    # 이 상태에서 FC layer를 통해 최종 토큰 개수 만큼 값을 반환하게 됩니다.
    # out: (batch_size, tok_size) : batch의 element별로 token에 대해 예측값을 반환
    out = self.fc1(out.squeeze(1))
    
    return out, hidden

In [151]:
def init_hidden(self, x):
  """
  레퍼런스 참고할 때 hidden_state 구조와 동일하게 되어야 합니다.
  여기서 보면 
  when batch_first=True (num_directional*num_layers, batch_size, hidden_size)
  """
  batch_size = x.size(0) if self.batch_first else x.size(1)
  h0 = torch.zeros(self.num_layers*self.num_directional, batch_size, self.hidden_size).to(device)
  c0 = torch.zeros(self.num_layers*self.num_directional, batch_size, self.hidden_size).to(device)
  
  return h0, c0

## Seq2Seq

In [152]:
class Seq2Seq(nn.Module) :

  def __init__(self, enc, dec) :

    super(Seq2Seq, self).__init__()
    assert enc.hidden_size == dec.hidden_size
    assert enc.num_layers == dec.num_layers

    self.encoder = enc
    self.decoder = dec

  def forward(self, enc_inp, dec_inp, use_teacher_force = True) :
    """
    enc_inp : (batch_size, enc_seq_len)
    dec_inp : (batch_size, dec_seq_len)
    """

    batch_size = enc_inp.size(0)
    enc_seq_len = enc_inp.size(1)
    dec_seq_len = dec_inp.size(1)
    # 최종 반환하는 것은 "토큰 별" score 값이니 이 점을 이용해서 token 개수 확인
    token_size =  self.decoder.fc1.out_features

    # decoder의 결과값은 출력 시퀀스에서 각 위치에 들어갈 token들의 확률(점수) 예측값 저장할 것입니다.
    # 그래서 구조는 (batch_size, seq_len, token_size)가 됩니다.
    # 즉 decoder는 step 마다 한 토큰씩 예측하게 되고 이를 총 seq_len만큼 진행해서 모으게 됩니다.
    # 이를 저장할 변수는 outputs
    # 이 때 seq_len은 decoder의 seq_len에 해당
    outputs = torch.zeros(batch_size, dec_seq_len, token_size)

    # Step 0: Encoder의 forward에 필요한 초기 hidden, cell state 계산
    enc_init_hidden = init_hidden(self.encoder, enc_inp)

    # Step 1: Encoder를 이용하여 context 벡터 계산
    # 중요한 점은 Encoder의 output이 아닌, hidden_state에 해당한다!!!
    _, context = self.encoder(enc_inp, enc_init_hidden)

    # Step 2: Decoder의 초기 입력값을 먼저 확인
    # 여기서 Decoder의 입력값이란 context vector 말고 단어들을 말합니다.
    # 그리고 첫 입력값은 <SOS>가 사용되어야 한다는 것
    # 이는 decoder의 input에서 첫번째 값에 해당하고 지금 상황에서는 앞서 사용한 \t을 알려주는 토큰이 사용되어야 합니다.
    # 그리고 Decoder의 인풋은 위에서 구현할 때  x: (batch_size, seq_len=1) 라고 했고 이 때 seq_len은 1이 됩니다.
    # 즉 이 구조를 맞춰서 넣어줘야 돌아가기에
    # (batch_size, dec_seq_len)의 차원을 맞춰야 하고, dec_seq_len=1이라서 마지막 차원에 unsqueeze 진행
    dec_inp_t = dec_inp[:, 0].unsqueeze(-1)

    # Step 3: Decoder를 이용하여 하나하나 계산에 사용
    # Encoder의 최종 리턴한 hidden state가 context 벡터고 이게 decoder입장에서보면 init_hidden에 해당합니다.
    # 또한 앞서 첫 입력값은 직접 할당했으니 전체 loop 시작은 1부터 하면 됩니다.
    # loop의 순서는 output 계산 --> 예측 --> 다음 입력값 업데이트
    # 이 순서라서 range는 1부터 시작하게 만들면 됩니다.
    dec_hidden = context
    for t in range(1, dec_seq_len):
      out, dec_hidden = self.decoder(dec_inp_t, dec_hidden)
      # 각 step별 결과물 = 해당 시점에서 각 토큰별 등장 확률(점수)값 기록
      outputs[:, t, :] = out

      if use_teacher_force:
        # teacher force를 사용하는 경우에는 실제 input에서 입력값을 넣어야 해서 decoder의 input값을 이용해서 업데이트를 합니다.
        # 그리고 모델이 돌아가기 위해서 차원도 차원 맞출 필요 존재힙니다.
        # 즉, indexing하게 되버리면 dec_inp차원과 맞지 않아서 unsqueeze 진행
        dec_inp_t = dec_inp[:, t].unsqueeze(-1)
      else:
        # keepdim을 통해 차원수 유지해줘서 구조를 동일하게 맞춘다.
        dec_inp_t = out.argmax(1, keepdim=True)

    return outputs

# Setup

In [153]:
# padding에 해당하는 token도 고려하기
enc_token_size = len(src_to_index)+1
dec_token_size = len(tar_to_index)+1

# Hyper-parameter!!
# RAM 메모리 이슈로 다운 되는 경우 때문에 줄였습니다.
input_size = 4
hidden_size = 4

In [154]:
encoder = Encoder(input_size, hidden_size, enc_token_size)
decoder = Decoder(input_size, hidden_size, dec_token_size)

model = Seq2Seq(encoder, decoder).to(device)

## 생략된 내용들

모델 생성하는 과정에서 초기값이나 등등 부수적인 부분들은 현재 과제 코드상에서 생략되었습니다. 모델 개선에 관심있으신 분들은 수행하셔도 좋습니다.

In [ ]:
# ~~~

# Train

In [155]:
def train(model, loader, optimizer, cri) :

  model.train()
  loss_ep = 0

  for enc_input, dec_input, dec_target in loader:
    optimizer.zero_grad()

    # 여기서 output은 seq2seq 모델의 결과값임을 유의해야합니다!
    # output: (batch_size, seq_len, token_size)
    output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=True)
    token_size = output.size(-1)

    # 지금 [문장, 문장, ...] 구조에서 문장들을 그냥 순서대로 이어 붙인 것
    # 이는 loss 계산을 쉽게 하기 위해 구조를 조정
    # output : (batch_size * seq_len, token_size)
    output = output.view(-1, token_size)

    # 위와 동일한 방식으로 label 값도 동일한 구조로 만들어주려고 view 실행
    # dec_target: (batch_size, dec_seq_len)
    # 즉 그냥 1열로 다 펴 버리면 CrossEntropyloss를 위한 구조 완성
    # target L (batch_size*dec_seq_len,)
    target = dec_target.view(-1)

    loss = cri(output, target)

    loss.backward()
    optimizer.step()

    loss_ep += loss.item()
    
  return loss_ep/len(loader)

# Evaluation

In [ ]:
import pandas as pd

In [156]:
def val(model, loader, cri) :

  # 평가할 땐 loss말고 예측한 결과도 저장할 것
  # 이것이 바로 result
  result = torch.Tensor()

  model.eval()
  loss_ep = 0

  with torch.no_grad() :

    for enc_input, dec_input, dec_target in loader:
      # output: (batch_size, seq_len, token_size)
      output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=False)

      # 예측 값을 저장
      decode_idx = output.argmax(2).detach().cpu()
      # 각 시점별 예측 결과를 저장
      result = torch.cat((result, decode_idx), dim=0)
  
      # 지금 [문장, 문장, ...] 구조에서 문장들을 그냥 순서대로 이어 붙인 것
      # loss 계산을 위한 조정
      # train과 동일한 이유 
      token_size = output.size(-1)
      output = output.view(-1, token_size)

      # 위와 동일한 방식으로 label 값도 조정
      # dec_target: (batch_size, dec_seq_len)
      # 즉 그냥 1열로 다 펴 버리면 CrossEntropyloss를 위한 구조 완성
      # target L (batch_size*dec_seq_len,)
      target = dec_target.view(-1)

      loss = cri(output, target)
      loss_ep += loss.item()
      
  return loss_ep/len(loader), result
  return loss_ep/len(loader), result

In [157]:
# 아래의 dictionary는 정수 인코딩 값을 넣으면 해당하는 원래 알파벳이 나옴
# padding도 처리해야하기 때문에 0은 그냥 ''으로 처리
# 이는 예측한 정수 인코딩 값을 실제 단어로 바꾸기 위한 것이라서 key, value를 바꿔주는 방식
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())
index_to_tar[0] = ''
index_to_src[0] = ''

In [158]:
def decode(model, sample):
  enc_input, dec_input, dec_output = sample
  enc_input = enc_input.unsqueeze(0)
  dec_input = dec_input.unsqueeze(0)
  
  model.eval()
  with torch.no_grad():
    output = model(enc_input.to(device), dec_input.to(device), use_teacher_force=False)
    decode_idx = output.argmax(2)

  sentence_inp = pd.Series(enc_input.squeeze(0).detach().cpu().numpy()).apply(lambda x: index_to_src[x])
  sentence_out = pd.Series(decode_idx.squeeze(0).detach().cpu().numpy()).apply(lambda x: index_to_tar[x])
  sentence_act = pd.Series(dec_output.detach().cpu().numpy()).apply(lambda x: index_to_tar[x])

  print(f"입력 문장 : {' '.join(sentence_inp.tolist())}")
  print(f"실제 문장 : {' '.join(sentence_act.tolist())}")
  print(f"예측 문장 : {' '.join(sentence_out.tolist())}")

In [159]:
import time
from tqdm import tqdm
from torch import optim

In [160]:
# 학습은 아래의 코드를 이용하여 진행

train_lodaer = DataLoader(train_dataset, batch_size=1024)
val_loader = DataLoader(val_dataset, batch_size=1024, shuffle=False)

optimizer = optim.Adam(model.parameters(), lr=1e-2)
criterion = nn.CrossEntropyLoss()

train_loss = []
val_loss = []

epochs = 10
for epoch in tqdm(range(epochs)):
  start = time.time()

  train_loss_ep = train(model,train_lodaer,optimizer, criterion)
  val_loss_ep, val_token = val(model, val_loader, criterion)

  train_loss.append(train_loss_ep)
  val_loss.append(val_loss_ep)
  if epoch % 2 == 0:
    print(f"Epoch : {epoch}")
    print(f"Train Loss : {train_loss_ep:.4f}")
    print(f"Val Loss : {val_loss_ep:.4f}")
    decode(model, val_dataset[100])


  0%|          | 0/10 [01:07<?, ?it/s]


KeyboardInterrupt: ignored

In [ ]:
# 학습을 진행한 다음, 모델 백업해놓기
torch.save(model.state_dict(), 'model_hw.pt')